In [1]:
from transformers import RobertaForCausalLM, AutoTokenizer, Trainer, TrainingArguments, DataCollatorForLanguageModeling, AutoModelForMaskedLM, AutoModelForCausalLM
import torch, datasets, sacremoses

device = ("cuda" if torch.cuda.is_available() else "cpu")

model_name = "flax-community/papuGaPT2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
model.config.pad_token_id = tokenizer.eos_token_id

## Data


In [2]:
def group_texts(examples, block_size=512):
    
    concatenated = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = (len(concatenated["input_ids"]) // block_size) * block_size

    result = {}
    for k, v in concatenated.items():
        result[k] = [v[i:i + block_size] for i in range(0, total_length, block_size)]
        
    return result

In [3]:
ds = datasets.load_dataset("text", data_files={
   "train": "pan_tadeusz_1_10.txt",
   "validation": "pan_tadeusz_11.txt",
   "test": "pan_tadeusz_12.txt",
})


def tokenize_function(examples):
   return tokenizer(examples["text"])


tokenized_datasets = ds.map(tokenize_function, batched=True, remove_columns=["text"])
tokenized_datasets = tokenized_datasets.map(group_texts, batched=True)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [4]:
import torch
import torch.nn.functional as F

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    logits = torch.tensor(logits)
    labels = torch.tensor(labels)
    
    shift_logits = logits[..., :-1, :].reshape(-1, logits.shape[-1])
    shift_labels = labels[..., 1:].reshape(-1)
    loss = F.cross_entropy(shift_logits, shift_labels)
    perplexity = torch.exp(loss).item()
    return {"perplexity": perplexity}

## Base check

In [5]:
model.eval()
prompt = "Jam jest Jacek"
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)

with torch.no_grad():
    outputs = model.generate(
        input_ids,
        max_length=200,
        do_sample=True,
        temperature=1.0,
        top_p=0.9
    )

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Generated text before training:\n", generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Generated text before training:
 Jam jest Jacek – szef działu produkcji filmowej w jednej z firm produkujących w Polsce. „Jest nam nie tylko przykro, że to co się dzieje w branży filmowej może być postrzegane wyłącznie przez pryzmat polityki i przemysłu filmowego, ale też dlatego, że to, co robimy w mediach, nie jest naszym osobistym interesem.”
„Z całym szacunkiem, ale nie rozumiem tego co robią ci wszyscy Ci z opozycji, a to, co robią media, nie tylko to, co robią politycy, nie jest polityką. To, co robią politycy, także to, co robią ludzie związani z branżą filmową, także to, co robią media, jest naszym interesem.”
„A może jednak jest tak jak mówi – każdy widzi co innego. Tak jest wszędzie. W TVP to są ludzie, którzy coś pokazują i coś się na nich natykają. Nie jest to na pewno film, który mógłby się obronić przed jakąkolwiek krytyką, nie jest to też film, który mógłby coś zmienić.”
„Z całą pewnością


## Training

In [6]:
from transformers import TrainerCallback

class GenerateTextCallback(TrainerCallback):
    def __init__(self, tokenizer, prompt="Jam jest Jacek", max_length=200):
        self.tokenizer = tokenizer
        self.prompt = prompt
        self.max_length = max_length

    def on_epoch_end(self, args, state, control, **kwargs):
        model = kwargs['model']
        model.eval()

        input_ids = self.tokenizer(self.prompt, return_tensors="pt").input_ids.to(model.device)
        
        with torch.no_grad():
            outputs = model.generate(
                input_ids,
                max_length=self.max_length,
                do_sample=True,
                top_p=0.9,
                temperature=1.0,
                repetition_penalty=1.2,
                num_return_sequences=1
            ) 
        
        generated_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        print(f"\n--- Generated text after epoch {state.epoch}:")
        print(generated_text)
        print("---------------------------\n")

In [7]:
from transformers import EarlyStoppingCallback

training_args = TrainingArguments(
    output_dir="./decoder-pan-tadeusz",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    num_train_epochs=10,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_steps=50,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="perplexity",
    greater_is_better=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[GenerateTextCallback(tokenizer),  EarlyStoppingCallback(early_stopping_patience=2)]
)


trainer.train()

C:\Users\litwi\AppData\Local\Temp\ipykernel_2768\1935341375.py:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss,Perplexity
1,No log,4.827408,124.885567
2,No log,4.747366,115.278801
3,4.685000,4.711594,111.228004
4,4.685000,4.692184,109.089920
5,4.343000,4.686679,108.490974
6,4.343000,4.685804,108.395981
7,4.170200,4.686109,108.429070
8,4.170200,4.687651,108.596245


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



--- Generated text after epoch 1.0:
Jam jest Jacek, że go tu przyprowadzi! – Głos jej i twarz jego były zamyślone. Poruszona pytała co się stało ze mną? — Mówił do mnie: Ja dziś z nim pójdę na wesele po to tylko żebym była weselsza nie jako córka mojej matki; a w domu moim niech by ojciec przykazał rodzicom moich braciom trzymać córkę u siebie lub jeżeli kto może dać komukolwiek o tym wiedzieć pod rękę ojca swojego albo córki syna mego… A ja mu przyrzekłem mówić głośno gdy będzie nas obchodził dzień jak mój brat...
— To on był królem Polski dzisiaj przez króla Jerzego II czyli Jana Kazimierza zwany - mówi panna Teodorowicz Ociepalska jeden spośród gości weselnych obok młodej panny siedzącej przed ołtarzem (to Helena), aby ją przywitać słowami pieśni "Poranny prorok" wyśpiewując swój wielki wiersz pt.: «Panie nasz miły Panie nasze». Ona także została matką chrzestną swego potomka Stanisława Wagińskiego zwanego „Wa
---------------------------



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...


--- Generated text after epoch 7.0:
Jam jest Jacek. On mi to wysłowił: «Jacek!» — od tych wszystkich osób, które nas znają jako gościnność nasza; a ja mu tylko na ten znak mówię po polsku z francuzkiem Francuzem językiem francuskim i ojczystym rapéjem w obozie (Sędzio znał go jeszcze)».Wracamy? Jakimż prawem nie można było się kłócić o Polskę czy inne rzeczy narodowe jak właśnie ją nazywać Rzeczpospolitą Polską?!Nareszcie doszliśmy ku granicom ojczyzny naszej dawnej…Znów my sami musieliśmy mówić pro ducato – Francuzi inaczej niż Polak we zwyczajuI często już tu był czas wspomnieć polską mowę :DSiedzę przy pudle bez celu bo mam dość siedzenia przez kilka minut(Śmiech pana Hrabiego).Usłyszałem że jakieś ruchy mają za mnąWięc wyjdę przed dom żeby pogadać co słychać u ludzi moich lub ich znajomychA gdzie bywam częściej czyli jestem gościem obcym dla mnie krajuPosłuchałem gościa lecz on zaczął
---------------------------



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



--- Generated text after epoch 8.0:
Jam jest Jacek).Oddał życie za to co było.Za karę,ze go nie znaja;Ale gdy ja byłem w niewoli(Odyseję)Stary został przy życiu jak gdyby nigdy nic:I o tym że mnie z Jackiem chcieli spalić rzekł ScyzorykiemKrukaHrabia Dobrzyński*…«Niedźwiedź» — powtórzył szlachcic-odcisnął pięść i spojrzał prosto na Sędziego».Teraz się odwracał Sędzia już był bliski ukropu lecz cóż powie do oczu Wodzowi?    Lecz ten pierwszy raz tak bardzo przestraszył sędzięGodząc myśli jąkałAż sędzia wpadł we wściekłośćBo czuł pogardę dla tego człowieka ,byrny zwierzańca!Przerażenie ścisnęło serce Hrana .Znowu chciał zemścić siekierą a zaraz znowu uciekaćWtem ujrzał dwóch mężów idących razem pod ramięRozprawiając nad losem gadadłaSokoła (Witka),Atakujący Soplice
---------------------------



There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=192, training_loss=4.329573790232341, metrics={'train_runtime': 508.8289, 'train_samples_per_second': 3.734, 'train_steps_per_second': 0.472, 'total_flos': 397163888640000.0, 'train_loss': 4.329573790232341, 'epoch': 8.0})